In [1]:
#Luca: We must tile all the WSI before proceeding with either segmentation or WSI-level. This notebook does the preprocessing 


#Segmentation Pipeline
    # Tile
    # Give tile name based on coordinate
    # Extract polygon annotation
    # Generate Masks based on Polygon Annotation
    # Find BG using ../saved_models/patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224)
    # Separate tiles into BG, Inf and Healthy folders
    
#WSI-level Pipeline
    # Tile
    # Remove BG tiles using ../saved_models/patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224)
        # and name them sequentially


In [2]:
import os
#import os.path
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import pyvips as Vips
from tqdm import tqdm
from utils import vips_utils, normalize
from torchvision import transforms, utils
import time
import torchvision.models as models
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from PIL import Image, ImageFile
import statistics
from typing import Optional, Tuple
import pylibczi
from pylibczi import CziScene
import czifile
from czifile import CziFile 
import xml.etree.ElementTree as ET
import argparse
import gc 
import psutil
import resource
import platform
import pickle
import xmltodict
import time
import matplotlib.path as mpath
from skimage.draw import polygon
import cv2
import copy
import shutil

import torch
import random
import torchvision
import torch.nn as nn
from skimage import io, transform



In [3]:
### PARAMETERS

TILE_SIZE = 512

WSI_DIR = '/cache/Inf_May23_dataset/'
SAVE_DIR = '/cache/S23_Infarct/patched_'+str(TILE_SIZE)+'/'
CZ_DIR = '/cache/S23_Infarct/annotation/'
MASK_DIR = '/cache/S23_Infarct/masks/'

SEGMENTATION_TILE_DIR = '/cache/S23_Infarct/seg_data_'+str(TILE_SIZE)+'/'
WSI_TILE_DIR = '/cache/S23_Infarct/wsi_level_data_'+str(TILE_SIZE)+'/'

tile_czi = True
tile_svs = True

In [4]:
if not os.path.exists(WSI_DIR):
    print("WSI folder does not exist, script should stop now")
else:
    if not os.path.exists(SEGMENTATION_TILE_DIR):
        print("Tile folder you provided us does not exist, being created now...")
        os.makedirs(SEGMENTATION_TILE_DIR)
        
    if not os.path.exists(WSI_TILE_DIR):
        print("Tile folder for WSI-level you provided us does not exist, being created now...")
        os.makedirs(WSI_TILE_DIR)

    if not os.path.exists(SAVE_DIR):
        print("Tile folder you provided us does not exist, being created now...")
        os.makedirs(SAVE_DIR)

    if not os.path.exists(CZ_DIR):
        print("Annotation folder you provided us does not exist, being created now...")
        os.makedirs(CZ_DIR)

    if not os.path.exists(MASK_DIR):
        print("Mask folder you provided us does not exist, being created now...")
        os.makedirs(MASK_DIR)

    print("Found WSI folder... ")
    wsi_slides = os.listdir(WSI_DIR)
    imagenames = sorted(wsi_slides)
    print("All WSIs in wsi_dir: ")
    print(imagenames)

Found WSI folder... 
All WSIs in wsi_dir: 
['NA-5029-16_HE.svs', 'NA-5029-17_HE.svs', 'NA-5029-18_HE.svs', 'NA5004-16_HE.svs', 'NA5004-17_HE.svs', 'NA5004-18_HE.svs', 'NA5007-16_HE.svs', 'NA5007-17_HE.svs', 'NA5007-18_HE.svs', 'NA5009-16_HE.svs', 'NA5009-17_HE.svs', 'NA5009-18_HE.svs', 'NA5031-16_HE.svs', 'NA5031-17_HE.svs', 'NA5031-18_HE.svs', 'NA5041-16_HE.svs', 'NA5041-17_HE.svs', 'NA5041-18_HE.svs', 'NA5045-16_HE.svs', 'NA5045-17_HE.svs', 'NA5045-18_HE.svs', 'NA5051-16_HE.czi', 'NA5051-17_HE.czi', 'NA5051-18_HE.czi', 'NA5057-16_HE.svs', 'NA5057-17_HE.svs', 'NA5057-18_HE.svs', 'NA5063-16_HE.czi', 'NA5063-17_HE.czi', 'NA5063-18_HE.czi', 'NA5077-16_HE.czi', 'NA5077-17_HE.czi', 'NA5077-18_HE.czi', 'NA5085-16_HE.czi', 'NA5085-17_HE.czi', 'NA5085-18_HE.czi', 'NA5089-16_HE.czi', 'NA5089-17_HE.czi', 'NA5089-18_HE.czi', 'NA5090-16_HE.czi', 'NA5090-17_HE.czi', 'NA5090-18_HE.czi', 'NA5091-16_HE.czi', 'NA5091-17_HE.czi', 'NA5091-18_HE.czi', 'NA5093-16_HE.czi', 'NA5093-17_HE.czi', 'NA5093-18_HE

In [ ]:
###### SEGMENTATION / WSI-level Pipeline - Tile


def grabCZI(path, verbose = False):
    img = czifile.imread(path)
    if verbose:
        print(img.shape)
        print(img)
    
    img = np.array(img, dtype = np.uint8)
    
    scenes = img.shape[0]
    time = img.shape[1]
    height = img.shape[2]
    width = img.shape[3]
    channels = img.shape[4]
    
    
    img = img.reshape((height, width, channels))
    if verbose:
        print(img)
        print(img.shape) 
        
    dtype_to_format = {
        'uint8': 'uchar',
        'int8': 'char',
        'uint16': 'ushort',
        'int16': 'short',
        'uint32': 'uint',
        'int32': 'int',
        'float32': 'float',
        'float64': 'double',
        'complex64': 'complex',
        'complex128': 'dpcomplex',
    }
    
    ###codes from numpy2vips
    height, width, bands = img.shape
    img = img.reshape(width * height * bands)
    vips = Vips.Image.new_from_memory(img.data, width, height, bands,
                                      dtype_to_format['uint8'])
    try: 
        del img, height, width, bands
        gc.collect()
    except: 
        pass
    
    return vips

print("Starting tiling....")
for imagename in tqdm(imagenames[:]):
    start = time.time()
    if imagename.split('.')[-1] == 'svs':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        vips_img = Vips.Image.new_from_file(WSI_DIR + imagename, level=0)
        
        print("Pre resize: ", vips_img.height, "x", vips_img.width)
        
        if NAID in ['NA5009-16_HE', 'NA5009-17_HE', 'NA5009-18_HE']:
            pass
            #vips_img = vips_img.resize(0.25)
        else:
            print("Resizing small case")
            vips_img = vips_img.resize(2)
            
        print("Post resize: ", vips_img.height, "x", vips_img.width)
            
        print("Loaded Image: " + WSI_DIR + imagename)

        if tile_svs:
        
            vips_utils.save_and_tile(vips_img, os.path.splitext(imagename)[0] \
                                     , SAVE_DIR, tile_size = TILE_SIZE)
            print("Done Tiling: ", WSI_DIR + imagename)
        
    elif imagename.split('.')[-1] == 'czi':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        try: 
            vips_img = grabCZI(WSI_DIR + imagename)
            print("Loaded Image: " + WSI_DIR + imagename)
            
            print("Pre resize: ", vips_img.height, "x", vips_img.width)
            
            if tile_czi:
        
                vips_utils.save_and_tile(vips_img, os.path.splitext(imagename)[0] \
                                         , SAVE_DIR, tile_size = TILE_SIZE)

                print("Done Tiling: ", WSI_DIR + imagename)
                del vips_img
                gc.collect()
                print("Finish Delete", WSI_DIR + imagename)
        except:
            print("Error in tiling")
        
        

    else:
        print("Skipped,", imagename, '. This file is either not .czi or .svs, or not the file assigned')
    
    try: 
        del vips_img 
        gc.collect()
    except:
        pass
    
    print("processed in ", time.time()-start," seconds")
    print("____________________________________________")

Starting tiling....


  0%|          | 0/69 [00:00<?, ?it/s]

Loading NA-5029-16_HE.svs  ......
Pre resize:  34033 x 45816
Resizing small case
Post resize:  68066 x 91632
Loaded Image: /cache/Inf_May23_dataset/NA-5029-16_HE.svs
/cache/S23_Infarct/patched_512/NA-5029-16_HE


  1%|▏         | 1/69 [01:13<1:22:47, 73.05s/it]

Done Tiling:  /cache/Inf_May23_dataset/NA-5029-16_HE.svs
processed in  73.0525529384613  seconds
____________________________________________
Loading NA-5029-17_HE.svs  ......
Pre resize:  33509 x 53784
Resizing small case
Post resize:  67018 x 107568
Loaded Image: /cache/Inf_May23_dataset/NA-5029-17_HE.svs
/cache/S23_Infarct/patched_512/NA-5029-17_HE


  3%|▎         | 2/69 [02:37<1:29:10, 79.85s/it]

Done Tiling:  /cache/Inf_May23_dataset/NA-5029-17_HE.svs
processed in  84.61224961280823  seconds
____________________________________________
Loading NA-5029-18_HE.svs  ......
Pre resize:  41428 x 47808
Resizing small case
Post resize:  82856 x 95616
Loaded Image: /cache/Inf_May23_dataset/NA-5029-18_HE.svs
/cache/S23_Infarct/patched_512/NA-5029-18_HE


  4%|▍         | 3/69 [04:08<1:33:06, 84.64s/it]

Done Tiling:  /cache/Inf_May23_dataset/NA-5029-18_HE.svs
processed in  90.33986592292786  seconds
____________________________________________
Loading NA5004-16_HE.svs  ......
Pre resize:  32738 x 43824
Resizing small case
Post resize:  65476 x 87648
Loaded Image: /cache/Inf_May23_dataset/NA5004-16_HE.svs
/cache/S23_Infarct/patched_512/NA5004-16_HE


In [ ]:
###### WSI-level Pipeline - Find BG using ./patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224) and name
# files sequentially

MODEL_SEG_DIR = './patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt'

seg_model = torchvision.models.resnet18(pretrained=True)
seg_model.fc = nn.Linear(512, 3)

checkpoint = torch.load(MODEL_SEG_DIR,map_location='cpu')
seg_model.load_state_dict(checkpoint['model_state_dict'])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
seg_model.to(device)


for NAID in os.listdir(SAVE_DIR):
    start = time.time()
    print("Processing ", NAID)
    if not os.path.exists(WSI_TILE_DIR+NAID):
        os.makedirs(WSI_TILE_DIR+NAID)
    
    seq_ct = 0
    for tile_folder in os.listdir(SAVE_DIR+NAID+'/0/'):
        for tile in os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder):
            
            seg_model.train(False)
            
            img = Image.open(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile)
            
            img_tensor = transforms.ToTensor()(img)
            img_tensor = transforms.Normalize(mean=[0.4409763317567454, 0.4016568471536302, 0.4988298669112181],
                             std=[0.31297803931100737, 0.2990562933047881, 0.33747493782548915])(img_tensor)
            img_tensor = torch.reshape(img_tensor,(1,3,512,512))
            img_tensor = img_tensor.cuda()
            
            predict = seg_model(img_tensor)
            preds = F.sigmoid(predict)
            _, indices = torch.max(predict.data, 1) # indices = 0:Background, 1:WM, 2:GM
            indices = indices.type(torch.uint8)
            running_seg =  indices.data.cpu()
            
            
            if running_seg == 0:
                shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, WSI_TILE_DIR+NAID+'/'+str(seq_ct)+'.jpg')
            else:
                pass
                #print("bg removed")
                
            seq_ct += 1
    print("processed in ", time.time()-start," seconds")
    print("____________________________________________")

In [ ]:
###### SEGMENTATION - Give tile name based on coordinate

print("About to change names to add coordinates")
for case_folder in sorted(os.listdir(SAVE_DIR)):
    NAID = case_folder
    print('NAID: ', NAID)
    for tile_folder in sorted(os.listdir(SAVE_DIR+NAID+'/0/')):
        # folder_level == y axis distance determinant
        # file_level == x axis distance determinant
        y0 = int(tile_folder)*TILE_SIZE
        y1 = (int(tile_folder)+1)*TILE_SIZE
        #print("Y axis = (", y0,',',y1,')')
        for tile_file in sorted(os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder+'/')):
            x0 = int(tile_file.split('.')[0])*TILE_SIZE
            x1 = (int(tile_file.split('.')[0])+1)*TILE_SIZE
            #print("X axis = (", x0,',',x1,')')
            #print("Renaming ", SAVE_DIR+NAID+'/0/'+str(tile_folder)+'/'+tile_file)
            os.rename(SAVE_DIR+NAID+'/0/'+str(tile_folder)+'/'+tile_file, 
                      SAVE_DIR+NAID+'/0/'+str(tile_folder)+'/'+str(y0)+'_'+str(x0)+'_'+str(y1)+'_'+str(x1)+'_'+tile_file)
            #print("Renamed into ", SAVE_DIR+NAID+'/0/'+tile_folder+'/'+str(y0)+'_'+str(x0)+'_'+str(y1)+'_'+str(x1)+'_'+tile_file)

In [ ]:
###### SEGMENTATION - Extract polygon annotation

for imagename in tqdm(imagenames[:]):
    start = time.time()
    os.makedirs(CZ_DIR+NAID)
    if imagename.split('.')[-1] == 'svs':
        pass
        
    elif imagename.split('.')[-1] == 'czi':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        try: 
            czifile = pylibczi.CziFile(WSI_DIR + imagename, metafile_out = CZ_DIR + NAID + '.cz',use_pylibczi=True, verbose=True)
            czifile.read_meta()
            
            tree = ET.parse(CZ_DIR + NAID + '.cz') 
            root = tree.getroot() 
            
            tree.write(CZ_DIR + NAID + '.xml')
        except:
            print("Error in extracting annotation")
            
    print("processed in ", time.time()-start," seconds")
    print("____________________________________________")

In [ ]:
###### SEGMENTATION - Generate Masks based on Polygon Annotation

def create_polygon_mask(vertex_coordinates, file_dimensions):
    
    mask = np.zeros(file_dimensions,'uint8')
    rr, cc = polygon(vertex_coordinates[:,0],vertex_coordinates[:,1],mask.shape)
    mask[rr,cc] = 1
    
    return mask

for imagename in tqdm(imagenames[:]):
    start = time.time()
    if imagename.split('.')[-1] == 'svs':
        pass
# TODO: FIND HOW TO REMOVE ANNOTATIONS FROM SVS XML

#         NAID = imagename.split('.')[0]
#         print("Loading", imagename, " ......")
#         vips_img = Vips.Image.new_from_file(WSI_DIR + imagename, level=0)
        
#         dimension_dict[NAID] = [vips_img.height, vips_img.width]
        
#         print("Loaded Image: " + WSI_DIR + imagename)

        
    elif imagename.split('.')[-1] == 'czi':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        try: 
            vips_img = grabCZI(WSI_DIR + imagename)
            print("Loaded Image: " + WSI_DIR + imagename)
            
            dimension = [vips_img.height, vips_img.width]
            
            doc = xmltodict.parse(open(CZ_DIR + NAID + '.xml', 'r', encoding='utf-8').read())
            
            coords = []
            for xy in doc['GraphicsDocument']['Elements']['Bezier'][0]['Geometry']['Points'].split(' '):
                coord = [int(float(xy[1])),int(float(xy[0]))]
                
                ### LUCA NOTE:
                    # I just assumed y was [1] and x was [0], this must be checked
                
                coords.append(coord)
                
            coords = np.asarray(coords,dtype=np.uint8)
            
            mask = create_polygon_mask(coords, dimension)
            
            mask = np.packbits(mask,axis=None)
            print("Binarized mask")

            np.save(MASK_DIR+NAID+'.npy',mask) 
            
            print("processed in ", time.time()-start," seconds")
            print("____________________________________________")

        except:
            print("Error in generating masks from polygon")
        
        
    else:
        print("Skipped,", imagename, '. This file is either not .czi or .svs, or not the file assigned')
        
    del vips_img,coords,mask
    gc.collect()

In [ ]:
###### SEGMENTATION - Find BG using ../saved_models/patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224)

MODEL_SEG_DIR = './patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt'

seg_model = torchvision.models.resnet18()
seg_model.fc = nn.Linear(512, 3)

checkpoint = torch.load(MODEL_SEG_DIR)
seg_model.load_state_dict(checkpoint['model_state_dict'])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
seg_model.to(device)


for NAID in os.listdir(SAVE_DIR):
    os.makedirs(SEGMENTATION_TILE_DIR+NAID+'/BG/')
    for tile_folder in os.listdir(SAVE_DIR+NAID+'/0/'):
        for tile in os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder):
            seg_model.train(False)
            
            img = Image.open(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile)
            
            img_tensor = transforms.ToTensor()(img)
            img_tensor = transforms.Normalize(mean=[0.4409763317567454, 0.4016568471536302, 0.4988298669112181],
                             std=[0.31297803931100737, 0.2990562933047881, 0.33747493782548915])(img_tensor)
            img_tensor = torch.reshape(img_tensor,(1,3,512,512))
            img_tensor = img_tensor.cuda()
            
            predict = seg_model(img_tensor)
            preds = F.sigmoid(predict)
            _, indices = torch.max(predict.data, 1) # indices = 0:Background, 1:WM, 2:GM
            indices = indices.type(torch.uint8)
            running_seg =  indices.data.cpu()
            
            
            if running_seg == 0:
                shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/BG/'+tile)
            else:
                pass
                #print("bg removed")

In [ ]:
###### SEGMENTATION - Separate tiles into BG, Inf and Healthy folders

for NAID in os.listdir(SAVE_DIR):
    proceed = True
    
    if os.path.isfile(WSI_DIR+NAID+'.svs'):
        vips_img = Vips.Image.new_from_file(WSI_DIR+NAID+'.svs', level=0)
    elif os.path.isfile(WSI_DIR+NAID+'.czi'):
        vips_img = grabCZI(WSI_DIR+NAID+'.czi')
    else:
        print("Could not find WSI - it is not czi nor svs")
        proceed = False
    
    if proceed:
        os.makedirs(SEGMENTATION_TILE_DIR+NAID+'/Heal/')
        os.makedirs(SEGMENTATION_TILE_DIR+NAID+'/Inf/')
        
        img_dim = [vips_img.height,vips_img.width]
        mask = np.load(MASK_DIR+NAID+'.npy')
        print("Loaded mask at ", MASK_DIR+NAID+'.npy' ,", about to process...")

        im_size = img_dim[0]*img_dim[1]

        mask = np.unpackbits(mask,count=im_size).reshape(img_dim).view(bool)
        mask = mask.view(np.uint8)
        print("Unbinarized mask")
        
        
        for tile_folder in os.listdir(SAVE_DIR+NAID+'/0/'):
            for tile in os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder):

                #file naming convention --> y0 x0 y1 x1
                y0 = tile.split('_')[0]
                x0 = tile.split('_')[1]
                y1 = tile.split('_')[2]
                x1 = tile.split('_')[3]
                
                if sum(sum(masky[y0:y1,x0:x1])) >= (512/3):
                    shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/Inf/'+tile)
                else:
                    shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/Heal/'+tile)
            
            
    del vips_img, mask
    gc.collect()